In [1]:
!pip install AugmentedSocialScientist pandas numpy wordcloud matplotlib spacy nltk
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 38.3 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


REMARQUE : BESOIN D'AVOIR FAIT TOURNE DESCRIPTION ANNOTATIONS POUR CREER LE FICHIER AVEC LES PHRASES !

In [1]:
import numpy as np
import random
import torch
import sklearn
random_seed = 1968
# Set random seed for NumPy
np.random.seed(random_seed)

# Set random seed for Python built-in random module
random.seed(random_seed)

# Set random seed for PyTorch
torch.manual_seed(random_seed)

# Set random seed for Scikit-learn
sklearn.utils.check_random_state(random_seed)


RandomState(MT19937) at 0x7FB7A99EC240

In [3]:
import pandas as pd

current_annotations_file = "../../data/intermediate/annotations/personne_withSentence_1400annotations_2024-06-03_ActiveTigger.csv"

annotations_files = pd.read_csv(current_annotations_file, dtype = {'sentence_id':'int', 
'Personne':'str','sentence':'str'}).set_index('sentence_id')

recat = {
    "Corps humain":"Corps humain",
    "Genre/espèce humaine":"Genre/espèce humaine",
    "Individu ou sujet" : "Individu ou sujet",
    "Personne à protéger":"Personne droits et devoirs",
    "Personne responsable et autonome":"Personne droits et devoirs",
    "Personne relationnelle":"Personne relationnelle",
    "Sentiment humain":"Genre/espèce humaine",
    "Poubelle" : "Poubelle"
}

frecat = lambda x: recat[x]
annotations_files['recat']=annotations_files["Personne"].apply(frecat)


annotations = annotations_files[~(annotations_files["recat"]=="Poubelle")]

annotations['label'] = annotations['recat']

annotations['text'] = annotations['sentence']

annotations.sample(5)

/tmp/ipykernel_33036/3703100089.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annotations['label'] = annotations['recat']
/tmp/ipykernel_33036/3703100089.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annotations['text'] = annotations['sentence']


,Personne,sentence,recat,label,text
sentence_id,,,,,
1190093,Personne responsable et autonome,"En revanche, \nleur rapidité et leur relative ...",Personne droits et devoirs,Personne droits et devoirs,"En revanche, \nleur rapidité et leur relative ..."
870236,Personne relationnelle,Refus par un tiers 8 \n \nIl est de nombreuses...,Personne relationnelle,Personne relationnelle,Refus par un tiers 8 \n \nIl est de nombreuses...
1210148,Sentiment humain,"Elle est une réalité \nhumaine, qui tient à la...",Genre/espèce humaine,Genre/espèce humaine,"Elle est une réalité \nhumaine, qui tient à la..."
860089,Personne responsable et autonome,"Ainsi l’ensemble de ces inconvénients qui, pou...",Personne droits et devoirs,Personne droits et devoirs,"Ainsi l’ensemble de ces inconvénients qui, pou..."
1130224,Corps humain,21415 »30 Voir l’avis n°112 du CCNE « Une réf...,Corps humain,Corps humain,21415 »30 Voir l’avis n°112 du CCNE « Une réf...


# Bert classification multicat

In [4]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

from torch import cuda

cuda.get_device_name(0)

'NVIDIA A2'

In [5]:
from sklearn.model_selection import train_test_split
 
train_data, test_data = train_test_split(annotations, train_size=0.80, random_state=random_seed, stratify=annotations["Personne"])

In [6]:
from AugmentedSocialScientist.models import Camembert

bert = Camembert()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


There are 1 GPU(s) available.
We will use GPU 0: NVIDIA A2


In [7]:
batch_size = 16
train_loader = bert.encode(train_data.text.values,
train_data.label.values, batch_size=batch_size)

  0%|          | 0/1110 [00:00<?, ?it/s]

  0%|          | 0/1110 [00:00<?, ?it/s]

label ids: {'Corps humain': 0, 'Genre/espèce humaine': 1, 'Individu ou sujet': 2, 'Personne droits et devoirs': 3, 'Personne relationnelle': 4}


In [8]:
test_loader = bert.encode(test_data.text.values,
test_data.label.values, batch_size=batch_size)

  0%|          | 0/278 [00:00<?, ?it/s]

  0%|          | 0/278 [00:00<?, ?it/s]

label ids: {'Corps humain': 0, 'Genre/espèce humaine': 1, 'Individu ou sujet': 2, 'Personne droits et devoirs': 3, 'Personne relationnelle': 4}


In [9]:
scores = bert.run_training(
    train_loader,
    test_loader,
    n_epochs=5,
    lr=2e-5,
    random_state=random_seed,
    save_model_as='multicat'
    )

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/opt/mamba/lib/python3.12/site-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



======== Epoch 1 / 5 ========
Training...
  Batch    40  of     70.    Elapsed: 0:00:49.

  Average training loss: 1.53
  Training took: 0:01:25

Running Validation...

  Average test loss: 1.34
  Validation took: 0:00:07
                            precision    recall  f1-score   support

              Corps humain       0.60      0.81      0.69        48
      Genre/espèce humaine       0.65      0.43      0.52        51
         Individu ou sujet       0.00      0.00      0.00        50
Personne droits et devoirs       0.44      0.94      0.60        84
    Personne relationnelle       0.00      0.00      0.00        45

                  accuracy                           0.50       278
                 macro avg       0.34      0.44      0.36       278
              weighted avg       0.36      0.50      0.40       278


======== Epoch 2 / 5 ========
Training...


/opt/mamba/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/mamba/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/mamba/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/mamba/lib/python3.12/site-packages/sklearn/m

  Batch    40  of     70.    Elapsed: 0:00:49.

  Average training loss: 1.22
  Training took: 0:01:25

Running Validation...

  Average test loss: 1.08
  Validation took: 0:00:07
                            precision    recall  f1-score   support

              Corps humain       0.80      0.81      0.80        48
      Genre/espèce humaine       0.73      0.78      0.75        51
         Individu ou sujet       0.71      0.10      0.18        50
Personne droits et devoirs       0.52      0.90      0.66        84
    Personne relationnelle       0.80      0.36      0.49        45

                  accuracy                           0.63       278
                 macro avg       0.71      0.59      0.58       278
              weighted avg       0.69      0.63      0.59       278


======== Epoch 3 / 5 ========
Training...
  Batch    40  of     70.    Elapsed: 0:00:49.

  Average training loss: 0.99
  Training took: 0:01:25

Running Validation...

  Average test loss: 0.97
  Validat

In [10]:
categories = ["Corps humain","Genre/espèce humaine","Individu ou sujet",
            "Personne relationnelle","Personne droits et devoirs"]
pd.DataFrame({
    "category" : categories,
    "precision" : scores[0],
    "recall" : scores[1],
    "f1-score" : scores[2],
    "support" : scores[3]
}).set_index("category").to_csv("./models/multicat/scores.csv")

In [11]:
pd.DataFrame({
    "category" : categories,
    "precision" : scores[0],
    "recall" : scores[1],
    "f1-score" : scores[2],
    "support" : scores[3]
}).set_index("category")

,precision,recall,f1-score,support
category,,,,
Corps humain,0.780000,0.812500,0.795918,48
Genre/espèce humaine,0.722222,0.764706,0.742857,51
Individu ou sujet,0.656250,0.420000,0.512195,50
Personne relationnelle,0.622449,0.726190,0.670330,84
Personne droits et devoirs,0.659091,0.644444,0.651685,45


In [12]:
import os
import zipfile

def zip_folder(folder_path, zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                # Create a relative path for the file inside the zip archive
                file_path = os.path.join(root, file)
                relative_path = os.path.relpath(file_path, folder_path)
                zipf.write(file_path, relative_path)
    print(f"Folder '{folder_path}' successfully zipped to '{zip_file_path}'")

# Example usage
folder_path = '/models/multicat'
zip_file_path = 'multicat.zip'
zip_folder(folder_path, zip_file_path)


Folder '/models/multicat' successfully zipped to 'multicat.zip'


# Prediction

In [13]:
base_shuffled = pd.read_pickle("base_shuffled.pkl")
base_shuffled.sentence_id = base_shuffled.sentence_id.apply(int)

base_shuffled.set_index('sentence_id', inplace= True)

base_shuffled.head(5)

,sentence_text,sentence,personne,humain,individu,Titre,Date,president,rapporteurs,membres_gt,theme,saisine,divergence,CNPEN,tableaux et graphiques,nb_mots,number_sentences,nb_pages,sentence_index,num
sentence_id,,,,,,,,,,,,,,,,,,,,
770722,Avec l’accumulation de grandes quantités de do...,Le principe de la noncommercialisation du corp...,False,True,False,Problèmes éthiques posés par les collections d...,2003-03-20,Sicard (00-08),"Hélène Gaumont-Prat,Nicole Questiaux,Gérard Orth","Nicole Baumann,Anne Cambon-Thomsen,Monique Can...",Société,False,0.0,0.0,1,22196,730,51,722,77
930350,Le critère du consentement : le consentement p...,Dans la mesure où le matériel biologique brut ...,True,False,False,Commercialisation des cellules souches humaine...,2006-06-22,Sicard (00-08),Monique Canto-Sperber,"Anne Cambon-Thomsen,Hélène Gaumont-Prat,Chanta...","Don, consentement",False,0.0,0.0,0,22028,703,40,350,93
1070205,"Notons, en outre, que si l’on considérait que ...","Or, le Comité estime pr écisément que la catég...",True,False,False,Avis sur les problèmes éthiques liés aux diagn...,2009-10-15,Grimfeld (09-11),"Pierre Le Coz,Dominique Stoppa-Lyonnet","Annick Alpérovitch,Joëlle Belaisch-Allart,Fréd...",Procréation,False,0.0,0.0,0,14877,520,28,205,107
180058,"Dans cette perspective, on concluera que légit...","De fondation, en vertu même des principes qui ...",True,False,False,Etat des études conduites par le Comité concer...,1989-12-15,Bernard (83-91),NaN,NaN,Génétique et Diagnostic,False,0.0,0.0,0,4177,142,8,58,18
1301177,"Qui fautil informer, quand et com ment ? Comme...",Comment respecter le droit \nde la personne de...,True,False,False,"Données massives et santé : Etat des lieux, pr...",2019-06-29,Delfraissy (16-),"Laure Coulombel,Pierre Delmas-Goyon","Gilles Adda,Thomas Bourgeron,Jean-Marie Delaru...",Société,True,0.0,0.0,0,44296,1373,94,1177,130


In [16]:
import numpy as np
pred_data = pd.DataFrame()
pred_data["text"] = base_shuffled["sentence"].dropna()
pred_loader = bert.encode(pred_data.text.values, batch_size=batch_size)
pred_proba = bert.predict_with_model(pred_loader, model_path='./models/multicat')
pred_data['pred_label'] = np.argmax(pred_proba, axis=1)
pred_data['pred_proba'] = np.max(pred_proba, axis=1)

  0%|          | 0/7753 [00:00<?, ?it/s]

  0%|          | 0/7753 [00:00<?, ?it/s]

  0%|          | 0/485 [00:00<?, ?it/s]

label ids: {'Corps humain': 0, 'Genre/espèce humaine': 1, 'Individu ou sujet': 2, 'Personne droits et devoirs': 3, 'Personne relationnelle': 4}


In [17]:
pred_data["pred_proba"].describe()

count    7753.000000
mean        0.541151
std         0.100822
min         0.248559
25%         0.462002
50%         0.550237
75%         0.609154
max         0.714482
Name: pred_proba, dtype: float64

In [18]:
pd.set_option('display.max_colwidth', None)  # Use None to show all the text, or set a large number

pred_data

,text,pred_label,pred_proba
sentence_id,,,
770722,"Le principe de la noncommercialisation du corps humain, en vigueur dans\nde nombreux pays et que reconnaissent la France et l’Allemagne interdit\npour des raisons qui dépassent largement le problème des biobanques, de\nmettre des éléments de son corps à la disposition d’autrui contre\nrémunération.",0,0.681766
930350,"Dans la mesure où le matériel biologique brut est d onné par une personne et peut faire \nl’objet d’une utilisation indifférenciée, la questi on du consentement de la personne source à \nl’utilisation de ce qui vient de son propre corps e t à l’exploitation commerciale dont un \nproduit de son corps serait éventuellement l’objet, se pose de façon aiguë.",0,0.707401
1070205,"Or, le Comité estime pr écisément que la catégorie de \n« prévention » est inappropriée lorsqu’elle vise à engager une personne testée à \ninformer sa parentèle des risques encourus par un e nfant non encore né.23 Dans tous les \ncas de figure, selon le CCNE, l’expression « préven ir une naissance » (à laquelle l’actuel \nCode de la santé publique aboutit à donner crédit) semble excessivement paradoxale.",3,0.559114
180058,"De fondation, en vertu même des principes qui ont inspiré sa composition, le Comité réunit\ndes personnes d'opinions, de croyances, de sensibilités, d'expériences les plus diverses.",3,0.520383
1301177,Comment respecter le droit \nde la personne de ne pas savoir ?,3,0.543636
...,...,...,...
1301154,"Dans le \ncontexte des données génomiques, se pose aussi la q uestion de la restitution à \nla personne de l’information résultant du traitemen t de ses données.",2,0.376378
30263,"Par ailleurs, le principe, qui est en France fortement reconnu, que l'utilisation de\nproduits du corps humain ne peut avoir un caractère lucratif, conduit à réserver à certaines\npersonnes le droit d'être intermédiaires.",0,0.514760
80015,"- Les inquiétudes, comme les espérances, que fait naître la maîtrise de la reproduction\nhumaine, suscitent, en effet, le désir de soumettre l'exercice de cette maîtrise à des normes\net/ou à des contrôles.",1,0.619694


In [20]:
labels_id = {0:'Corps humain', 1:'Genre/espèce humaine', 2:'Individu ou sujet', 3:'Personne droits et devoirs', 4:'Personne relationnelle'}
frecat = lambda x: labels_id[x]

pred_data['pred_Personne'] = pred_data['pred_label'].apply(frecat)

In [22]:
pred_data.to_pickle("predictions_camemBERT_1400annotations_5classes_2024-06-04.pkl")